In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2896, 거래소 1530, 코스닥 1366
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2896 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 265,655,299,000,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 55,182,579,267,000
삼성전자우 상장주식수: 822,886,700, 시가총액 30,446,807,900,000
현대차 상장주식수: 213,668,187, 시가총액 26,815,357,468,500
NAVER 상장주식수: 164,813,395, 시가총액 24,309,975,762,500
현대모비스 상장주식수: 95,306,694, 시가총액 23,350,140,030,000
LG화학 상장주식수: 70,592,343, 시가총액 23,154,288,504,000
셀트리온 상장주식수: 128,336,328, 시가총액 20,084,635,332

쎄노텍 상장주식수: 40,158,175, 시가총액 81,119,513,500
세코닉스 상장주식수: 11,803,281, 시가총액 81,088,540,470
해덕파워웨이 상장주식수: 73,696,405, 시가총액 81,066,045,500
YTN 상장주식수: 42,000,000, 시가총액 81,060,000,000
대교우B 상장주식수: 19,426,990, 시가총액 81,010,548,300
배럴 상장주식수: 7,818,100, 시가총액 80,917,335,000
세원정공 상장주식수: 10,000,000, 시가총액 80,900,000,000
동운아나텍 상장주식수: 13,618,450, 시가총액 80,893,593,000
씨큐브 상장주식수: 10,138,184, 시가총액 80,801,326,480
대원전선 상장주식수: 68,107,171, 시가총액 80,706,997,635
까스텔바쟉 상장주식수: 6,596,700, 시가총액 80,479,740,000
이더블유케이 상장주식수: 7,380,944, 시가총액 80,452,289,600
동일금속 상장주식수: 7,000,000, 시가총액 80,150,000,000
화일약품 상장주식수: 14,410,958, 시가총액 80,124,926,480
상신브레이크 상장주식수: 21,471,450, 시가총액 79,873,794,000
에스에프씨 상장주식수: 43,981,834, 시가총액 79,827,028,710
데브시스터즈 상장주식수: 11,204,370, 시가총액 79,775,114,400
삼천리자전거 상장주식수: 13,273,577, 시가총액 79,774,197,770
에이텍 상장주식수: 8,260,000, 시가총액 79,709,000,000
기가레인 상장주식수: 52,276,120, 시가총액 79,459,702,400
스맥 상장주식수: 27,955,979, 시가총액 79,394,980,360
에이스토리 상장주식수: 9,318,240, 시가총액 79,391,404,800
휴스틸 상장주식수: 7,621,668, 시가총액 79,26

KODEX 국채선물10년인버스 상장주식수: 396,000, 시가총액 17,665,560,000
테라셈 상장주식수: 14,398,118, 시가총액 17,637,694,550
미래에셋 Q150 Core5 ETN 상장주식수: 2,000,000, 시가총액 17,590,000,000
신한 인버스 은 선물 ETN(H) 상장주식수: 2,000,000, 시가총액 17,460,000,000
제이웨이 상장주식수: 18,534,210, 시가총액 17,459,225,820
KINDEX 밸류대형 상장주식수: 2,700,000, 시가총액 17,388,000,000
TRUE 코리아프리미어 ETN 상장주식수: 2,000,000, 시가총액 17,370,000,000
KODEX 미국S&P고배당커버드콜(합성 H) 상장주식수: 1,800,000, 시가총액 17,262,000,000
QV 화학 TOP5 ETN 상장주식수: 2,000,000, 시가총액 17,190,000,000
삼성 증권 테마주 ETN 상장주식수: 2,000,000, 시가총액 17,110,000,000
대상우 상장주식수: 1,370,223, 시가총액 16,922,254,050
삼성 온라인쇼핑 테마주 ETN 상장주식수: 2,000,000, 시가총액 16,860,000,000
파인넥스 상장주식수: 24,015,495, 시가총액 16,834,861,995
QV 건설 TOP5 ETN 상장주식수: 2,000,000, 시가총액 16,820,000,000
TIGER 200 에너지화학 상장주식수: 1,460,000, 시가총액 16,636,700,000
엔에이치스팩12호 상장주식수: 7,510,000, 시가총액 16,484,450,000
지투하이소닉 상장주식수: 20,645,684, 시가총액 16,475,255,832
KBSTAR 채권혼합 상장주식수: 300,000, 시가총액 16,399,500,000
세동 상장주식수: 21,411,000, 시가총액 16,379,415,000
대신 인버스 2X 니켈선물 ETN(H) 상장주식수: 2,000,000, 

In [4]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [5]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    
    MarketTotal.append([name, item[0], listed, markettot])

In [6]:
import pandas as pd

In [7]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [8]:
pdMarketTotal.head()

,0,1,2,3
0,삼성전자,A005930,5969782,265655299000000
1,SK하이닉스,A000660,728002365,55182579267000
2,삼성전자우,A005935,822886700,30446807900000
3,현대차,A005380,213668187,26815357468500
4,NAVER,A035420,164813395,24309975762500


In [9]:
pdMarketTotal.loc[(
    (pdMarketTotal[3] > 1000000000000) &
    (pdMarketTotal[0].apply(lambda x: x.startswith("KODEX")))
)]

,0,1,2,3
44,KODEX 200,A069500,221300000,5726137500000
91,KODEX 레버리지,A122630,226400000,2480212000000
143,KODEX 단기채권,A153130,13605000,1378186500000
167,KODEX 코스닥150 레버리지,A233740,170400000,1162980000000
177,KODEX 종합채권(AA-이상)액티브,A273130,9970000,1091216500000


In [10]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization".split(",")

In [11]:
pdMarketTotal.to_csv("../data/market_capitalization.csv", index=None)